# Managing Products
Here we will outline the basic steps to create a new raster `Product` with associated `Images`. This example will also cover the basics of `Namespaces` and `Organizations` for managing access to your newly created `Product`.

This guide is meant to serve as an introduction to managing raster data. For a more in depth overview of all possible options please see the [Catalog Guide](https://docs.descarteslabs.com/guides/catalog.html#managing-products) located in the Documentation. 

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import (
    Image,
    DataType,
    Product,
    Resolution,
    ResolutionUnit,
    SpectralBand,
)

## Creating and updating a product
The only required attributes for a `Product` are a unique id and a name. 

A `Product ID` should always have the following structure:

    `my-org-id:my-product-id`

You can retrieve your `Organization` by the `Auth API`:

In [ ]:
org = dl.auth.Auth().payload["org"]
org

Since you may have other users within your `Organization` who have run this same example, let's take it one step further and append our default `Namespace` to our `Product ID`. Each `Descartes Labs` account has a unique default `Namespace` ID.

In [ ]:
namespace = dl.auth.Auth().namespace
namespace

In [ ]:
pid = f"{org}:{namespace}:guide-example-product"
pid

Now we can create our new `Product`:

In [ ]:
product = Product.get_or_create(pid)
product.name = "Example RGB Product"
product.tags = ["examples"]
product.save()
product

## Creating bands
Before adding any `Images` to a `Product` you must create `Bands` that declare the structure of the data shared among all images in a product.

The `Band` defines where its data is found in the files attached to images in the product: In the below example, `band_index = 0` indicates that `Red` is the first band in the image file, and that band is expected to be represented by unsigned 16-bit integers (`DataType.UINT16`). Note also that if we were to set `file_index = 1` it would expect a _second Image_ in that position to represent that `Band`. 

We'll start with a simple `SpectralBand`. Here, we have a `GeoTIFF` located under the path `data/rgb.tif` which contains 3 bands. We will create one `SpectralBand` per channel here, for `Red`, `Green`, and `Blue`, including supported wavelength properties.

In [ ]:
# These values are in nanometers (nm)
band_info_dict = {
    "red": {"wavelength_nm_min": 650, "wavelength_nm_max": 680},
    "green": {"wavelength_nm_min": 542.5, "wavelength_nm_max": 577.5},
    "blue": {"wavelength_nm_min": 457.5, "wavelength_nm_max": 522.5},
}

In [ ]:
for i, (name, info) in enumerate(band_info_dict.items()):
    band = SpectralBand(name=name, product=product)
    band.data_type = DataType.UINT16
    band.data_range = (0, 10000)
    band.display_range = (0, 4000)
    band.resolution = Resolution(unit=ResolutionUnit.METERS, value=30.0)
    band.band_index = i
    band.file_index = 0
    band.wavelength_nm_min = info["wavelength_nm_min"]
    band.wavelength_nm_max = info["wavelength_nm_max"]
    band.save()
    print(f"Saved: {band.id}")

See the [`Bands`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/band.html#bands) Documentation page for more information about other band types not covered in this example.

## Uploading Images - File Paths
`image.upload()` uploads imagery to a `Product` via a list of specified file paths. This returns an `ImageUpload`. Images are uploaded and processed asynchronously, so they are not available in the catalog immediately. With `upload.wait_for_completion()` we wait until the upload is completely finished:

In [ ]:
# Set any attributes that should be set on the uploaded images
image = Image(product=product, name="image1")
# Set acquired date
image.acquired = "2023-06-15"
# Set metadata
image.cloud_fraction = 0.1
# Do the upload
image_path = "data/rgb.tif"
upload = image.upload(image_path)
upload.wait_for_completion()
upload.status

## Uploading Images - Numpy Arrays
You can also call [`Image.upload_ndarray`](https://docs.descarteslabs.com/examples-gallery/plot_create_product.html?#upload-ndarray-to-new-product) if you have your pixels in-memory. Note you will need to manually specify `raster_info` as that is not information is not contained within a numpy array.

## Access Control
By default only the creator of a product can read and modify it as well as read and modify the bands and images in it. To share access to a product with others you can modify its access control lists (ACLs):

In [ ]:
product.readers = ["org:pga-tour"]
product.writers = ["email:jane.doe@descarteslabs.com", "email:john.daly@gmail.com"]
product.save()

What the above just did:
* Give read access to the whole `pga-tour` organization. All users in that organization can now find the product and retrieve it's imagery.
* Give write access to two specific users identified by email. These two users can now update the product and add new images to it. 

### Transfer ownership
Transferring ownership of a product to a new user requires cooperation from both the previous owner and the new owner and is a two-step effort. The first step is for the previous owner to add the new owner to the product:

Just a reminder that you cannot use the `email:` variant as an owner. You will have to request the user's `default Namespace` from the new owner and use that instead. (You can find your user id in the profile drop-down on [iam.descarteslabs.com](iam.descarteslabs.com) or through `dl.auth.Auth().namespace`).

The second step is for the new owner to remove the previous owner:


For further information on access control please see the [Sharing Resources](https://docs.descarteslabs.com/guides/sharing.html) guide.

## Deleting Bands and Products
Any catalog objects (`Products`, `Bands`, and `Images`) can be deleted using the `delete` method:

In [ ]:
product.delete()

#### *NOTE:* You ran into an error, didn't you!
A `Product` can _only be deleted if it doesn’t have any associated Bands or Images_. Because the `Product` we created still has images and bands we must first delete those related objects. 

There is a convenience method to delete all bands and images in a product. Be careful as this may delete a lot of data and can’t be undone!

In [ ]:
status = product.delete_related_objects()

This kicks off a job that deletes bands and images in the background. You can wait for this to complete and then delete the product:

In [ ]:
if status:
    status.wait_for_completion()
product.delete()